In [1]:
import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.graph_objs as go
import pandas as pd
import os

In [2]:
#Gathering stats for each tickers
stats_folder = "outputs/stats"
available_tickers = [f.replace("_stats.csv", "") for f in os.listdir(stats_folder) if f.endswith(".csv")]

In [3]:
#Creating the dashboard
app = dash.Dash(__name__)
app.title = "AI Trading Dashboard"

In [4]:
# == Layout ==
#Main structure
app.layout = html.Div(style={
    'backgroundColor': '#004d4d',
    'color': 'white',
    'padding': '30px',
    'fontFamily': 'Arial'
}, children=[
    html.H1("AI Trading Dashboard", style={
        'textAlign': 'center',
        'marginBottom': '30px',
        'color': 'white',
        'fontWeight': 'bold'
    }),
    
#Dropdown to choose tickers
    html.Div([
        dcc.Dropdown(
            id='ticker-dropdown',
            options=[{'label': t, 'value': t} for t in available_tickers],
            value=available_tickers[0],
            style={
                'backgroundColor': 'white',
                'color': '#004d4d',
                'width': '300px',
                'marginBottom': '20px'
            }
        )
    ], style={'textAlign': 'left'}),

#Graph for model vs Buy & Hold
    dcc.Graph(id='strategy-graph', style={'height': '400px'}),

#Boxes for statistics
    html.Div(id='statistics-box', style={
        'display': 'flex',
        'flexWrap': 'wrap',
        'gap': '20px',
        'marginTop': '20px',
        'justifyContent': 'center'
    }),

#Titel for trade-log
    html.H3("Trade-log", style={'marginTop': '40px', 'textAlign': 'left', 'color': 'white'}),

#Trade-log table
    html.Div(id='trade-table')
])

In [5]:
# == Callback == Updates the dashboard when a new ticker is chosen

@app.callback(
    [Output('strategy-graph', 'figure'),    #Updates the graph
     Output('statistics-box', 'children'),  #Updates the boxes with statistics
     Output('trade-table', 'children')],    #Updates the trade-log
    [Input('ticker-dropdown', 'value')]     #Updates the dropdown
)

def update_dashboard(ticker):

#Loads data for the chosen ticker
    signal_path = f"outputs/signals/{ticker}_signals.csv"
    stats_path = f"outputs/stats/{ticker}_stats.csv"
    trades_path = f"outputs/trades/{ticker}_trades.csv"

    df = pd.read_csv(signal_path, index_col=0, parse_dates=True)
    stats = pd.read_csv(stats_path)
    trades = pd.read_csv(trades_path)

    #Changing 1/-1 to Long/Short
    trades['position'] = trades['position'].map({1: 'Long', -1: 'Short'}).fillna('Neutral')

    #Changing the return to %
    trades['return'] = trades['return'].apply(lambda x: f"{x*100:+.2f}%")

    #Changing name on the columns
    trades.rename(columns={
    'entry_date': 'Entry Date',
    'exit_date': 'Exit Date',
    'position': 'Position',
    'entry_price': 'Entry Price',
    'exit_price': 'Exit Price',
    'return': 'Return'
    }, inplace=True)

    
#Creates the graph for the new data
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df.index, y=df['CumulativeStrategyReturn'],
        mode='lines', name='Strategi', line=dict(color='cyan')
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=df['CumulativeReturn'],
        mode='lines', name='Buy & Hold', line=dict(color='orange')
    ))

#Updating the graph
    fig.update_layout(
        title=f"{ticker} – Model vs Buy & Hold",
        template='plotly_dark',
        plot_bgcolor='#002b2b',
        paper_bgcolor='#002b2b',
        font=dict(color='white'),
        margin=dict(t=40, b=40)
    )

#Function to create the boxes with statistics
    def metric(label, value, suffix=""):
        return html.Div([
            html.H4(label, style={'marginBottom': '5px'}),
            html.P(f"{value:.2f}{suffix}", style={'fontSize': '24px', 'color': '#00ccff'})
        ], style={
            'padding': '10px',
            'backgroundColor': '#006666',
            'borderRadius': '10px',
            'minWidth': '140px',
            'textAlign': 'center'
        })

#Creating the boxes        
    stat_components = [
        metric("Return", stats['total_return_strategy'].iloc[0], "×"),
        metric("Max Drawdown", stats['max_drawdown'].iloc[0] * 100, "%"),
        metric("Sharpe Ratio", stats['sharpe_ratio'].iloc[0]),
        metric("Trades", stats['num_trades'].iloc[0]),
        metric("Win rate", stats['win_rate'].iloc[0] * 100, "%"),
        metric("Buy & Hold", df['CumulativeReturn'].iloc[-1], "×")
    ]

#Creating a layout for the trade-log    
    trade_table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in trades.columns],
        data=trades.to_dict('records'),
        style_table={"overflowX": "auto", 'maxHeight': '400px', 'overflowY': 'scroll'},
        style_header={
            'backgroundColor': '#004d4d',
            'fontWeight': 'bold',
            'color': 'white'
        },
        style_cell={
            'backgroundColor': '#e6ffff',
            'color': '#002b2b',
            'padding': '10px',
            'textAlign': 'center',
            'fontFamily': 'monospace'
        },
        page_size=15,
        style_data_conditional=[
    {
        'if': {'column_id': 'Return', 'filter_query': '{Return} contains "+"'},
        'color': 'green',
        'fontWeight': 'bold'
    },
    {
        'if': {'column_id': 'Return', 'filter_query': '{Return} contains "-"'},
        'color': 'red',
        'fontWeight': 'bold'
    }
]
    )

 
    return fig, stat_components, trade_table

In [7]:
#Starting the server
if __name__ == '__main__':
    app.run(debug=True, port=8051)